In [1]:
from sklearn.metrics import classification_report, cohen_kappa_score
from osgeo import gdal
from pathlib import Path

gdal.UseExceptions()

In [2]:
def read_raster(filePath: str):
    dataset = gdal.Open(filePath)

    # im_proj = dataset.GetProjectionRef()
    # im_geotrans = dataset.GetGeoTransform()
    # print('im_proj:')
    # print(im_proj)
    # print('im_geotrans:')
    # print(im_geotrans)

    nXSize = dataset.RasterXSize
    nYSize = dataset.RasterYSize
    im_data = dataset.ReadAsArray(0, 0, nXSize, nYSize)
    return {
        'name': Path(filePath).stem,
        'mat': im_data,
        'nodata': dataset.GetRasterBand(1).GetNoDataValue()
    }

In [3]:
def show_compare(file1: str, file2: str, nodata1=None, nodata2=None):
    img1 = read_raster(file1)
    img2 = read_raster(file2)

    if img1['mat'].shape != img2['mat'].shape:
        raise ValueError('images are in different shapes')

    if nodata1 != None:
        img1['nodata'] = nodata1
    if nodata2 != None:
        img2['nodata'] = nodata2

    print(f'image1 nodata: {img1["nodata"]}')
    print(f'image2 nodata: {img2["nodata"]}')

    mask1 = (img1['mat'] != img1['nodata'])
    print(f'image1 valid nums: {mask1.sum()}')
    mask2 = (img2['mat'] != img2['nodata'])
    print(f'image2 valid nums: {mask2.sum()}')
    mask = (mask1 & mask2)
    print(f'intersect mask nums: {mask.sum()}')

    vec1 = img1['mat'][mask]
    vec2 = img2['mat'][mask]

    print('classification report:')

    kappa = cohen_kappa_score(vec1, vec2)
    print(f'kappa: {kappa:.4f}')

    cr = classification_report(vec1, vec2, digits=4)
    print(cr)

In [4]:
show_compare('img/source.tif', 'img/result.tif')

image1 nodata: 0.0
image2 nodata: 0.0
image1 valid nums: 2084361
image2 valid nums: 2084361
intersect mask nums: 2084361
classification report:
kappa: 0.8286
              precision    recall  f1-score   support

           1     0.6438    0.9825    0.7779     76128
           2     0.9085    0.9279    0.9181    795885
           3     0.5817    0.8962    0.7055     95238
           4     0.6784    0.8953    0.7719     68979
           5     0.9825    0.8600    0.9172   1048131

    accuracy                         0.8932   2084361
   macro avg     0.7590    0.9124    0.8181   2084361
weighted avg     0.9135    0.8932    0.8980   2084361

